<a href="https://colab.research.google.com/github/powerpower777/Group_T_Laptop_Project/blob/main/Group_T_mini_laptop_price_model_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ydata_profiling

Import Libraries needed

In [ ]:

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
from ydata_profiling import ProfileReport

In [ ]:
import pandas as pd

# load dataset from the GitHub repository
df = pd.read_csv("https://raw.githubusercontent.com/powerpower777/Group_T_Laptop_Project/refs/heads/main/laptop_price.csv", encoding='latin-1')

Dataset Overview

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.columns

In [ ]:
df.sample(5)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include = "object")

The Data Cleaning And Analysis Part

In [ ]:
# Dropping the laptop_ID
df = df.drop(columns= "ï»¿laptop_ID")
df.head()

In [ ]:
# Changing then columns name to make it easily readable
df = df.rename(columns = {"Price_euros": "Price", "OpSys": "Operating_System"})

In [ ]:
# Removing the duplicate samples
df = df.drop_duplicates()
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# Filling in the null value with the most frequent value (mode) for 'Gpu' and median for 'Weight'
df['Gpu'] = df['Gpu'].fillna(df['Gpu'].mode()[0])
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)
df['Weight'] = df['Weight'].fillna(df['Weight'].median())

df.isnull().sum()

In [ ]:
# The value count of columns
for column in df.columns:
  print(f"{df[column].value_counts()} \n")

In [ ]:
# The countplot for all non-numeric columns
import matplotlib.pyplot as plt
import seaborn as sns

for column in df.select_dtypes(include= "object"):
 plt.figure(figsize = (20,5))
 sns.countplot(data= df, x =column)
 plt.title(f"Count Plot For {column}".capitalize())
 plt.xticks(rotation = 90)
 plt.show()

In [ ]:
# The Barplot for all non-numeric columns by Price

for column in df.select_dtypes(include= "object"):
 plt.figure(figsize = (20,5))
 sns.barplot(data= df, x =column, y= "Price")
 plt.title(f"Bar Plot For {column} Vs Price in Euros(£)".capitalize())
 plt.xticks(rotation = 90)
 plt.show()

In [ ]:
df.info()

In [ ]:
import pandas as pd

# Cleaned Dataset
df.to_csv('cleaned_dataset.csv', index=False)
from google.colab import files
files.download('cleaned_dataset.csv')

The Creation And Training Models

In [ ]:
x = df.drop(columns = ["Price", "Inches", "Weight"], axis=1 ) # Dropping columns that are not needed
y = df["Price"] # The target variable

In [ ]:
from sklearn.model_selection import train_test_split as split

x_train, x_test, y_train, y_test = split(x, y, test_size= 0.15 , random_state= 8)

In [ ]:
print(x_train.shape, y_train.shape)

In [ ]:
# Identifying categorical and numeric columns
categorical_columns = x.select_dtypes(include=['object']).columns
numerical_columns = x.select_dtypes(exclude=['object']).columns

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Transforming using encode categorical and scale numeric
preprocessor = ColumnTransformer(
    transformers=[
        ('category', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ('number', StandardScaler(), numerical_columns)
    ]
)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

# Pipeline: preprocessing + model
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [ ]:
# The Train
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions
predictions = model.predict(x_test)

# Evaluate model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

Creating data report using Ydata

In [ ]:
    import sys
    !{sys.executable} -m pip install -U ydata-profiling[notebook]
    !pip install jupyter-contrib-nbextensions

In [ ]:
from ydata_profiling import ProfileReport

report = ProfileReport(df, title='Group T Project Report',explorative=True)

In [ ]:
report.to_file("Group T Project Report.html")

In [ ]:
from google.colab import files
files.download("Group T Project Report.html")